# First let's install the Folium package

Lets google "Install folium package python"

https://python-visualization.github.io/folium/installing.html

In [3]:
#!pip install folium #run this line if using Folium for the first time

In [4]:
# Now that we have folium install, let's import our packages into the environment
import pandas as pd
import folium
from folium.plugins import HeatMap

# Now let's find out a fun variable to map

The requirements is that we need a table that is granular at the observation level and that has both (lat, lon).

We could begin by searching in:
* https://data.boston.gov/
* https://data.somervillema.gov/


In [5]:
# Let's load the data from the Json
json_address = 'https://data.somervillema.gov/resource/q3yh-mp87.json'
df = pd.read_json(json_address)
df.head(2)

,address,amountpaid,applicant,applicantaddress,applicantcitystzip,applicationdate,closedate,expirationdate,file,issuedate,latitude,longitude,permit,permitamount,permittype,permittypedetail,projectname,status
0,13 LINCOLN ST,110.0,"Lindmark's Plumbing & Heating co inc .,",13 Gove Rd,BILLERICA MA 01821,2019-08-29T00:00:00.000,NaN,2020-02-29T00:00:00.000,19-016388,2019-08-29T00:00:00.000,42.386021,-71.081261,G19-000563,110.0,Residential - Existing,Gas,2 new gas lines for multi unit house,Issued
1,375 REVOLUTION DR,197.0,Stanley Security,30 Commerace Way,Woburn MA 01801,2019-08-29T00:00:00.000,NaN,2020-02-29T00:00:00.000,19-016422,2019-08-29T00:00:00.000,42.392796,-71.081200,E19-001348,197.0,Commercial,Electrical,Low voltage Security System,Issued


In [6]:
# First create a list to give to Folium
heat_df = df[['latitude', 'longitude']]
heat_df = heat_df.dropna(axis=0, subset=['latitude','longitude'])
heat_data = [[row['latitude'],row['longitude']] for index, row in heat_df.iterrows()]

In [7]:
# We now create a map object and populate it with points
map = folium.Map(location=[42.38, -71.09], zoom_start=14, tiles='Stamen Terrain')
HeatMap(heat_data, radius=15).add_to(map)
map

# Lets play around with some of the arguments we can pass to the map
* tiles 
* zoom_start
* radius

In [37]:
# Now let's do a very disgusting dataset that requires some cleaning

In [8]:
json_address2 = 'https://data.somervillema.gov/resource/xs7t-pxkc.json'
df2 = pd.read_json(json_address2)
df2.head(2)

,:@computed_region_8cbr_52pd,issue_description,issue_type,location,neighborhood_district,secondary_issue_type,street_address,submitter,ticket_closed_date_time,ticket_created_date_time,ticket_id,ticket_last_updated_date_time,ticket_status
0,16551.0,Sewer,Manhole repair,"{'latitude': '42.379819', 'longitude': '-71.10...",Ward 2,Service Requests,165 Beacon St,43095,2015-07-02 14:51:00,2015-07-02 12:18:00,400579,2015-07-02 14:51:00,Closed
1,16881.0,ISD Health,Rats,"{'latitude': '42.390608', 'longitude': '-71.09...",Ward 4,Service Requests,11 Maple Ave,0,2015-07-23 00:16:00,2015-07-07 09:34:00,401523,2015-07-23 00:16:00,Closed


In [9]:
df2.issue_description.value_counts()

Traffic and Parking          234
Water                        195
DPW Highway                  184
ISD Health                   109
DPW Buildings and Grounds     87
DPW Sanitation                65
Sewer                         48
DPW Lights and Lines          30
Parking                       17
OSPCD Admin                   12
Animal Control                10
Constituent Services           9
Name: issue_description, dtype: int64

In [11]:
df3 = df2[df2['issue_description'] == 'Traffic and Parking']
df3.head(2)

,:@computed_region_8cbr_52pd,issue_description,issue_type,location,neighborhood_district,secondary_issue_type,street_address,submitter,ticket_closed_date_time,ticket_created_date_time,ticket_id,ticket_last_updated_date_time,ticket_status
10,16552.0,Traffic and Parking,T&P Miscellaneous,"{'latitude': '42.4013888', 'longitude': '-71.1...",Ward 6,Service Requests,21 Walker St,0,2015-07-22 09:39:00,2015-07-22 09:39:00,405749,2015-07-22 09:39:00,Closed
23,16552.0,Traffic and Parking,Illegal parking,"{'latitude': '42.3957061', 'longitude': '-71.1...",Ward 6,Service Requests,Grove St,17753,2015-08-27 15:20:00,2015-08-27 14:14:00,416405,2015-08-27 15:20:00,Closed


In [12]:
# We need to transform the disgusting variable into something workable
df3.location.head(2)

10    {'latitude': '42.4013888', 'longitude': '-71.1...
23    {'latitude': '42.3957061', 'longitude': '-71.1...
Name: location, dtype: object

In [13]:
df4 = pd.DataFrame(df3['location'].values.tolist(), index=df3.index)
df4.head()

,human_address,latitude,longitude
10,"{""address"": ""21 Walker St"", ""city"": ""Somervill...",42.4013888,-71.1186435
23,"{""address"": ""Grove St Somerville"", ""city"": """",...",42.3957061,-71.1204966
24,"{""address"": ""98 Elm St"", ""city"": ""Somerville"",...",42.3894709,-71.117085
38,"{""address"": ""45 Broadway"", ""city"": ""Somerville...",42.3865842,-71.0796899
46,"{""address"": ""0 Kent St"", ""city"": ""Somerville"",...",42.3837584,-71.1097916


In [14]:
df4[['latitude', 'longitude']].head()

,latitude,longitude
10,42.4013888,-71.1186435
23,42.3957061,-71.1204966
24,42.3894709,-71.117085
38,42.3865842,-71.0796899
46,42.3837584,-71.1097916


In [15]:
# First create a list to give to Folium
heat_df2 = df4[['latitude', 'longitude']]
heat_df2 = heat_df2.dropna(axis=0, subset=['latitude','longitude'])
heat_data2 = [[row['latitude'],row['longitude']] for index, row in heat_df2.iterrows()]

# We now create a map object and populate it with points
map2 = folium.Map(location=[42.38, -71.09], zoom_start=14, tiles='Stamen Terrain')
HeatMap(heat_data2, radius=15).add_to(map2)
map2

In [16]:
# Whoops! Something is wrong, our latitude and longitude aren't numbers!!

# Let's fix that
df4 = df4[['latitude', 'longitude']].astype(float)

In [17]:
# Let's try that again
heat_df2 = df4[['latitude', 'longitude']]
heat_df2 = heat_df2.dropna(axis=0, subset=['latitude','longitude'])
heat_data2 = [[row['latitude'],row['longitude']] for index, row in heat_df2.iterrows()]
map2 = folium.Map(location=[42.38, -71.09], zoom_start=14, tiles='Stamen Terrain')
HeatMap(heat_data2).add_to(map2)
map2

Also, make sure to look at the Folium documentation:

* https://python-visualization.github.io/folium/modules.html
* https://python-visualization.github.io/folium/plugins.html

class folium.plugins.HeatMap(data, name=None, min_opacity=0.5, max_zoom=18, max_val=1.0, radius=25, blur=15, gradient=None, overlay=True, control=True, show=True, **kwargs)